In [ ]:
# Import system modules
import arcpy
from arcpy import env
from arcpy.sa import *

In [2]:
# Read station catalogue file
root_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs"
in_out_folder = root_folder + "\\Streamflow_Sts_Drainage_Areas"
fc = in_out_folder + "\\Stations_Catalogue.shp" 

In [ ]:
hydrosheds = root_folder + "\\HydroSHEDS"
flow_dir_folder = hydrosheds + "\\FlowDirection"
flow_acc_folder = hydrosheds + "\\FlowAccumulation"

# Set overwrite option
env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

In [ ]:
# Make a layer from the feature class
arcpy.management.MakeFeatureLayer(fc, "selected_station")

snap_constant = 0.0046296296 # Change according to spatial resolution (approx. 500 m)     
Extent = "MAXOF"
factors = {"07374525": 0.5, "23": 0.2, "17050001": 0.25, "1662100": 0.2, "2181900": 0.7, "A4261110": 0.1, "6742900": 1.6, "42055": 0.8}

with arcpy.da.SearchCursor(fc, ["ID", "BASIN", "CONTINENT"]) as cursor:

    for row in cursor:

        print("Processing basin: " + row[1] + " - Drainage area of station: " + str(row[0]))

        # Select each station
        print("\tSelecting corresponding station")
        arcpy.SelectLayerByAttribute_management("selected_station", "NEW_SELECTION", "ID = '" + row[0] + "'")

        fdir_name = row[2] + "_dir_3s"
        facc_name = row[2] + "_acc_3s"
        fdir_grid = flow_dir_folder + "\\" + fdir_name + "\\" + fdir_name +'.tif'
        facc_grid = flow_acc_folder + "\\" + facc_name + "\\" + facc_name +'.tif'
        snapped_pour_point = in_out_folder + "\\snapped_pour_point_" + str(row[0]) +  "_" + row[1] + ".shp"
        watershed = in_out_folder + "\\Cathment_" + str(row[0]) +  "_" + row[1] + ".shp"
        snap_distance = snap_constant * factors[row[0]] # The resulting distance value represents the distance to the location/pixel where the pour point will be snapped

        # Process: Snap Pour Point (Snap Pour Point) (sa)
        with arcpy.EnvManager(extent=Extent):
            print("\tSnapping pour point")
            pour_point_raster = arcpy.sa.SnapPourPoint("selected_station", facc_grid, snap_distance, "FID")

        print("\tConverting snapped pour raster to point")
        # Process: Raster to Point (Raster to Point) (conversion)
        arcpy.conversion.RasterToPoint(pour_point_raster, snapped_pour_point, "VALUE")

        print("\tDelineating watershed")
        # Process: Watershed (Watershed) (sa)
        watershed_raster = arcpy.sa.Watershed(fdir_grid, pour_point_raster, "VALUE")

        print("\tConverting watershed raster to polygon")
        # Process: Raster to Polygon (Raster to Polygon) (conversion)
        arcpy.conversion.RasterToPolygon(watershed_raster, watershed, "SIMPLIFY", "VALUE", "SINGLE_OUTER_PART", None)

arcpy.CheckInExtension ('Spatial')
print("DONE!!")
